## Introduction

Emotion detection from text is an important task in natural language processing (NLP) that helps machines understand human feelings expressed through written language.

By fine-tuning pretrained transformer models like DistilBERT on labeled emotion datasets, I build a classifier that automatically identify emotions such as sadness, joy, anger, and fear from sentences.

This technology has real-world applications across many fields.

1. In customer service, emotion detection enables chatbots and support agents to respond empathetically to frustrated or happy customers.

2. In social media monitoring, it helps companies gauge public sentiment about products or events.

3. Mental health platforms use emotion recognition to monitor users’ emotional wellbeing and provide timely support.

4. Emotion detection is useful in marketing, content recommendation, and even human-computer interaction to create more personalized experiences.

In this notebook, I fine-tuned a transformer model for emotion classification, evaluated its performance, and tested it on various sample sentences.

## Installations

In [ ]:
# installs or updates the required librarires and dependencies
!pip install --quiet --upgrade transformers datasets huggingface_hub bertviz gcsfs fsspec umap-learn

## Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import datasets
from datasets import load_dataset
from huggingface_hub import list_datasets
from transformers import AutoModelForSequenceClassification, \
AutoTokenizer, Trainer, TrainingArguments, pipeline
import torch
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, f1_score

from sklearn.metrics import classification_report, confusion_matrix

import warnings
warnings.filterwarnings('ignore')

## Loading the Dataset

In [ ]:
emotions_dataset = load_dataset("dair-ai/emotion")

In [ ]:
emotions_dataset

## Data Preprocessing & EDA

In [ ]:
emotions_dataset.set_format(type = 'pandas')

In [ ]:
df_train = emotions_dataset['train'][:]

In [ ]:
df_train.head()

In [ ]:
classes = emotions_dataset['train'].features['label'].names
classes

In [ ]:
# applying class names
df_train['label_name'] = df_train['label'].apply(lambda x: classes[x])

In [ ]:
df_train.head()

In [ ]:
# counts
label_name_counts = df_train['label_name'].value_counts().sort_values(ascending=False)

# Horizontal bar graph
label_name_counts.plot(kind='barh', figsize=(8,5))
plt.title('Count of Each Label Name')
plt.xlabel('Count')
plt.ylabel('Label Name')
plt.show()

In [ ]:
# count words per tweet
df_train["Words per tweet"] = df_train['text'].str.split().apply(len)

In [ ]:
df_train.head(2)

In [ ]:
# box plot showing distribution of words in the dataset
sns.boxplot(data=df_train, y='label_name', x ='Words per tweet')
plt.title("Distribution of Words Per tweet")
plt.show()

## Loading the Model

In [ ]:
model_checkpoint = 'distilbert/distilbert-base-uncased'

## Tokenization

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
text = 'I am finetuning a transformer model'
encoded_text = tokenizer(text)
encoded_text

In [ ]:
emotions_dataset.reset_format()

In [ ]:
def tokenizing(batch):
  temp = tokenizer(batch['text'], padding=True, truncation=True)
  return temp

In [ ]:
# tonizing the train data
tokenizing(emotions_dataset['train'][:1])

In [ ]:
# tokenizing train, eval and test datasets
emotions_encoded = emotions_dataset.map(tokenizing, batched=True, batch_size=None)

In [ ]:
emotions_encoded, emotions_dataset

## Fine Tuning Process

In [ ]:
num_labels = len(classes)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# loading model, adding output
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels).to(device)

In [ ]:
# Training Arguments
batch_size = 62
model_name = 'distiled-bert-finedtuned-emotions'

training_args = TrainingArguments(
    output_dir = model_name,
    learning_rate = 2e-5,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    num_train_epochs = 2,
    weight_decay = 0.01,
    eval_strategy = 'epoch',
    disable_tqdm = False
)

In [ ]:
# Evaluates how well the model is performing after every epoch
def compute_metrics(pred):
  labels = pred.label_ids  # true labels
  preds = pred.predictions.argmax(-1) # predicted labels, retruns index of the highest score
  f1 = f1_score(labels, preds, average = 'weighted')
  acc = accuracy_score(labels, preds)
  return {'accuracy': acc, 'f1': f1}

In [ ]:
# Model trainer
trainer = Trainer(
    model = model,
    args = training_args,
    compute_metrics = compute_metrics,
    train_dataset = emotions_encoded['train'],
    eval_dataset = emotions_encoded['validation'],
    tokenizer = tokenizer
)

In [ ]:
# Training the Model
trainer.train()

## Evaluating Model Performance

In [ ]:
# Extract eval_loss and eval_accuracy from logs
eval_losses = [log['eval_loss'] for log in trainer.state.log_history if 'eval_loss' in log]
eval_accuracies = [log['eval_accuracy'] for log in trainer.state.log_history if 'eval_accuracy' in log]

print(eval_losses)
print(eval_accuracies)


In [ ]:
preds_outputs = trainer.predict(emotions_encoded['test'])
preds_outputs.metrics

In [ ]:
y_predicted_labels = np.argmax(preds_outputs.predictions, axis=1)

In [ ]:
emotions_encoded

In [ ]:
y_true_labels = emotions_encoded['test'][:]['label']

In [ ]:
print(y_predicted_labels[:5])
print(y_true_labels[:5])

In [ ]:
print(classes)
print()

# Classification Report
cr = classification_report(y_true_labels, y_predicted_labels)
print(cr)

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_true_labels, y_predicted_labels)

# Visulizing
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
def predict_emotion(text):
  # tokenize input
  inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)

  # Send inputs to same device as model
  inputs = {k: v.to(model.device) for k, v in inputs.items()}

  # Get raw logits from the model
  with torch.no_grad():
      outputs = model(**inputs)

  logits = outputs.logits  # Extracts logits from the ouput

  predicted_index = np.argmax(logits.cpu().numpy(), axis=1)[0] # moves logits to cpu, converts it to numpy array, returns index with highest score

  return classes[predicted_index]

In [ ]:
predict_emotion("She surprised me with a birthday party I didn’t expect!")

In [ ]:
test_sentences = [
    "I can't stop thinking about how everything could go wrong.",
    "I feel completely drained even though nothing major happened today.",
    "There's a strange calmness in me even though I know I should be worried.",
    "I'm happy for my friend, but I can't help feeling a little jealous too.",
    "I feel a deep sense of peace after confronting my fears.",
    "I'm frustrated because I know I can do better, but something holds me back.",
    "I’m excited about the opportunity, but the pressure is overwhelming.",
    "I feel guilty for being happy while others are suffering.",
    "I miss how things used to be — there’s a constant ache in my chest.",
    "Even in a room full of people, I feel invisible."
]

In [ ]:
# predictions on text sentences
for sentence in test_sentences:
  prediction = predict_emotion(sentence)
  print(f"Sentence: {sentence}\nPredicted Emotion: {prediction}\n")